Créé le 17 mars 2021

**Projet Tableau de Bord** 

**Groupe n°3 - Arnaques en ligne**

**Création des json pour l'insertion**

@authors:
- KIRED Nour Elhouda



#### Import libraries

In [2]:
########## Module import ##########
# Goole Colab
%load_ext google.colab.data_table
from google.colab import files

# Fichiers
import json

# traitement texte /Nettoyage des données
import operator
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import collections
from tqdm import tqdm
from gensim.utils import simple_preprocess
import spacy
import gensim
import gensim.corpora as corpora


nltk.download('stopwords')
stopWords=stopwords.words('english')
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### Importation des données 

In [5]:
uploaded = files.upload()
data_dict : dict = json.loads(uploaded['clean_articles_5629.json'])

#### Description des données  importées
 - data_dict est un dictionnaire de dictionnaire
 - Les clés sont des string d'entiers allant de 0 à 5629
 - Les valeurs sont des dictionnaires resprésentant les données d'un article
 - Ces données sont 'url', 'abstract', 'author', 'keyword', 'title' et 'date'
 - Cas de 'author' : c'est un dictionnaire qui associe un nom d'auteur à sa liste d'affiliations

#### Description du notebook 
- **V0**: Recuperation des types d'arnaques à partir de l'abstract utilisant Kmeans -> Resultats non pertinents 
- **V1** :Recuperation des types d'arnaques à partir de l'abstract en utilisant  une LDA -> Resultats non pertinents 
- **V2** : Recuperation des types d'arnaques à partir des titres en utilisant  une LDA -> themes.json en sortie

## Traitement du texte

#### Partie fonctions

In [6]:

def cleanedWords(sentence : str)-> str:
    """
       Fonction qui supprime les caractères spéciaux
        In :
            texts : un texte
        Out :
            le texte sans caracteres speciaux 

    """
    return (re.sub("[^a-zA-Z0-9]"," ", sentence.lower()))


def remove_stopwords(texts : str) -> list:
    """
        Fonction qui supprime les mots vides du texte
        In :
            texts : 
        Out :
            le texte sans mots vides

    """
    return [word for word in simple_preprocess(texts) if word not in stopWords] 
        
def lemmatization(texts : list)-> list:
    """
        Fonction : Lemmatisation d'un texte

        In :
            texts : liste de mots
        Out :
            liste de mots lemmatisée 
    """
    texts : str =" ".join(texts)
    return [token.lemma_ for token in nlp(texts)]


#### Nettoyage du body

- Clusterisation par titre 

In [7]:
clean_text : list =[]
for (i,row)in tqdm(data_dict.items()):
    # enlever les caracteres speciaux
    data_words : str =cleanedWords(row['title'])
    # enlever les Stop Words
    data_words_nostops : list = [word for word in data_words.split() if word not in stopWords] 
    # effectuer une lemmatisation  
    data_lemmatized : list = lemmatization(data_words_nostops)
    clean_text.append(data_lemmatized)


100%|██████████| 5629/5629 [00:16<00:00, 345.31it/s]


In [8]:
# Creation d'un dictionnaire de tous les mots des titres
id2word : list = corpora.Dictionary(clean_text)
print("dict ok")

# creaction du corpus (texts) 
texts  : str = clean_text
print("text ok")

# vectorisation de nos mots 
corpus: list  = [id2word.doc2bow(text) for text in texts]
print("corpus ok")

del  texts

dict ok
text ok
corpus ok


## Création des thèmes

#### Entrainement d'un modele 

On a décidé de laisser deux themes :
- description des arnaques
- detection des fraudes

In [9]:
## entrainement d'un modele qui va nous permettre de clusteriser nos 
## articles par theme
lda_model = gensim.models.LdaModel(corpus,
                                   num_topics=2,
                                   id2word=id2word,
                                   random_state=100,
                                   alpha='auto')

del id2word

In [11]:
def format_topics_sentences(ldamodel, corpus):
    """"Documentation
      Parameters:
            ldamodel : notre modele entrainé pour clusteriser nos articles par theme
            corpus : texte des articles
      Out :
            chaque article est associé à son theme (creation d'une column dans notre df)

    """
    # Iinitialisation de notre output df
    sent_topics_df : pd.DataFrame = pd.DataFrame()
    # recupere le theme qui maximise le score pour chaque article
    nb_comm  : int = len(corpus)
    list_topic : list  = [int(sorted(lda_model[corpus][i], key=lambda x: (x[1]),
                             reverse=True)[0][0]) for i in tqdm(range(nb_comm))]
    sent_topics_df['Dominant_Topic'] = list_topic

    return(sent_topics_df)

In [12]:
df_topic_sents_keywords = format_topics_sentences(
    ldamodel=lda_model, corpus=corpus)

df_dominant_topic = df_topic_sents_keywords.reset_index()

df_dominant_topic.columns = ['id', 'Dominant_Topic']

del df_topic_sents_keywords,lda_model


100%|██████████| 5629/5629 [00:01<00:00, 3574.82it/s]


#### Mots les plus frequents par (theme/article)

**- Par article**

In [13]:
common_words_article : list =[]
for data in clean_text:
  vectorizer: TfidfVectorizer = TfidfVectorizer(stop_words=stopWords)
  try:
    X = vectorizer.fit_transform(list(data))
    vocabulary: list = vectorizer.vocabulary_
    common_words_article.append(sorted(vocabulary.items() ,key=operator.itemgetter(1), reverse=True )[:3])
  except:
    common_words_article.append([])

In [14]:
common_words_ : list =[str(i) for i in common_words_article]
common_words_article : pd.DataFrame = pd.DataFrame(common_words_,columns=['commons words/article_iter'])

In [25]:
df : pd.DataFrame = pd.concat([common_words_article,df_dominant_topic['Dominant_Topic']],axis=1)
df.sample(10)

,commons words/article_iter,Dominant_Topic
5164,"[('prospect', 3), ('policy', 2), ('identity', 1)]",0
3996,"[('use', 12), ('technique', 11), ('service', 10)]",1
1538,"[('social', 7), ('security', 6), ('privacy', 5)]",1
24,"[('trust', 5), ('scam', 4), ('nigerian', 3)]",0
2323,"[('culture', 2), ('community', 1), ('amp', 0)]",1
441,"[('wearable', 7), ('technology', 6), ('securit...",0
1862,"[('sector', 4), ('health', 3), ('global', 2)]",1
2466,"[('socio', 6), ('social', 5), ('mining', 4)]",1
1833,"[('systems', 12), ('psychology', 11), ('popula...",1
4932,"[('vector', 3), ('payment', 2), ('channel', 1)]",0


**- Par theme**

In [23]:
## recuperation de  tous les bodies pour chaque theme
new_data : dict = {}
for index, row in df.iterrows():
  y=row['Dominant_Topic']
  if y not in new_data.keys():
    new_data[y]=clean_text[index]
  else:
    new_data[y]+=clean_text[index]
print(new_data)

{0: ['419', 'scam', 'evaluation', 'cybercrime', 'criminal', 'code', 'nigeria', 'e', 'commerce', 'security', 'fraud', 'issue', 'protection', 'dakar', 'arts', 'festival', 'explore', 'transmedia', 'storytelling', 'method', 'document', 'internet', 'scam', 'anatomy', 'online', 'date', 'romance', 'scam', 'e', 'commerce', 'security', 'fraud', 'issue', 'protection', 'challenge', 'pose', 'scammer', 'online', 'support', 'group', 'deserve', 'undeserve', 'victim', 'scam', 'e', 'commerce', 'security', 'fraud', 'issue', 'protection', 'scam', 'everyone', 'psychologically', 'profile', 'cyberscam', 'victim', 'spam', 'base', 'scam', 'spam', 'base', 'scam', 'alchemy', 'fraud', 'investment', 'scam', 'precious', 'metal', 'mining', 'business', 'dear', 'friend', 'scam', 'trust', 'multimethod', 'analysis', 'nigerian', 'email', 'scam', 'smart', 'fraud', 'detection', 'framework', 'job', 'recruitment', 'measure', 'cost', 'cybercrime', 'consumer', 'scam', 'agency', 'theoretic', 'approach', 'personal', 'situationa

In [30]:
most_common_words_themes : dict = {}
for (index,abstract) in new_data.items():
  split_it=[(re.sub("[^a-zA-Z0-9]", "", w.lower())) for w in abstract if w not in stopWords]
  count : dict = collections.Counter(split_it) 
  most_occur : list = count.most_common(6) 
  most_common_words_themes[index]=most_occur
print(most_common_words_themes)

{0: [('online', 800), ('internet', 672), ('e', 506), ('security', 490), ('base', 386), ('technology', 338)], 1: [('fraud', 847), ('detection', 562), ('internet', 459), ('crime', 398), ('use', 396), ('social', 376)]}


In [19]:
types_dict : dict ={0:'detection des fraudes',
            1:'description des arnaques',
}

In [26]:
themes: dict ={}
for (i,row) in df.iterrows():
  themes[i]=types_dict[row['Dominant_Topic']]

{0: 'detection des fraudes', 1: 'detection des fraudes', 2: 'detection des fraudes', 3: 'detection des fraudes', 4: 'detection des fraudes', 5: 'description des arnaques', 6: 'description des arnaques', 7: 'description des arnaques', 8: 'detection des fraudes', 9: 'description des arnaques', 10: 'detection des fraudes', 11: 'description des arnaques', 12: 'detection des fraudes', 13: 'description des arnaques', 14: 'description des arnaques', 15: 'detection des fraudes', 16: 'detection des fraudes', 17: 'description des arnaques', 18: 'description des arnaques', 19: 'description des arnaques', 20: 'detection des fraudes', 21: 'description des arnaques', 22: 'description des arnaques', 23: 'detection des fraudes', 24: 'detection des fraudes', 25: 'description des arnaques', 26: 'description des arnaques', 27: 'description des arnaques', 28: 'description des arnaques', 29: 'description des arnaques', 30: 'detection des fraudes', 31: 'description des arnaques', 32: 'description des arnaqu

#### Sauvegrade des données

In [ ]:
with open('themes.json', 'w') as f:
    json.dump(themes, f)